# BDC API Quickstart


The following code demonstrates the different BDC python API capabilities. Under the hood the python API uses the BDC REST API. This is intended as an annotated quickstart guide. Feel free to borrow from it as a skeleton for your actual code.

We begin by defining some constants and helper functions. The `USERNAME` is your web portal username. The `API_KEY` is a REST API key which you can generate by navigating to Tools -> REST API Key Manager on the website and copying the generated key. `HOST` is the server the API will run requests against.

The `run_query_progress_checks` function will be used later to check how far along a query is at regular time intervals. The `get_task_id` function is used to search for a particular task number from all the task numbers available to the user which are returned by the API function `get_task_numbers`.


In [1]:
from bdc_api import BdcApi
import time

# TODO: Replace this with your username.
USERNAME = 'example_username'
# TODO: Replace this with your API key.
API_KEY = 'b65e8c7f-c288-4bd0-bad5-37826c5d620c'
HOST = 'https://minos.lbl.gov'

def run_query_progress_checks(query_id):
    """Run subsequent checks every 3 seconds to see if a query succeeded.

    Parameters:

        :query_id: Valid query ID. 

    Returns:

        - None. 

    Raises:

        - None. 
    """
    print ('Checking query progress every 3 seconds for query {0}'.format(query_id))
    query_progress = ''
    while query_progress != BdcApi.COMPLETE_QUERY:
        time.sleep(3)
        query_progress = api.check_query_progress(query_id)
        print ('Current query progress: {0}'.format(query_progress))    

def get_task_id(search_task, task_numbers):
    """Get a task ID from a return of `BdcApi.get_task_numbers`. 

    Parameters:

        :search_task: Name of task to search for. 
        :task_numbers: return value of `BdcApi.get_task_numbers`. 

    Returns:

        - Task ID of the task to search for or -1.

    Raises:

        - None. 
    """
    for key in task_numbers:
        if search_task in key:
            task_id = task_numbers[key]['ID']
            print ('Found desired task ID {0} in task {1}. ID is: {2}'.format(
                search_task, key, task_id))
            return task_id
    return -1


We now instantiate an instance of `BdcApi` which will be used throughout for all API calls. Note how it depends only on username, a REST API key and a hostname. We define a task we want to search for ('Data Management') and use our helper function to search the return of the API function `get_task_numbers` for the task ID we're after. We'll use this task ID to query the domains in the next step. 

In [9]:
api = BdcApi(username=USERNAME, api_key=API_KEY,
    hostname=HOST)
task_name_to_search = 'Data Management'
task_numbers = api.get_task_numbers()
print ('******************************')
task_id = get_task_id(task_name_to_search, task_numbers)

******************************
Found desired task ID Data Management in task Data Management / 3. ID is: 5b9ad6f064bb061520ff321d


The following call returns the domains for the task we are looking for using its task ID. For simplicity, we'll use all domains returned by this function. In a real use-case, you would down-select 1 or more domains as needed from the returned list.

In [10]:
domains_by_task = api.get_domains(task_ID=task_id)
domain_list = []
for task in domains_by_task:
    domain_list.extend(domains_by_task[task])
print ('******************************')
print ('Using all domains of this task, i.e: {0}'.format(domain_list))

******************************
Using all domains of this task, i.e: ['guidance', 'example code', 'images', 'metatdata', 'Presentations']


We can now query all datacollections that are within this task and that include the domains we're looking for. We then use the first collection name and get its files. Note that the 1st datacollection is chosen here for simplicity. In a real use-case, you would search this list for the datacollection you're after. 

In [11]:
desired_coll = api.get_datacollections(task_numbers=task_name_to_search,
    domains=domain_list)[0]
file_list_for_desired_coll = api.get_files(datacollections=desired_coll)[desired_coll]
print ('******************************')
print ('Requesting subset of files:')
print (file_list_for_desired_coll[0:2])

******************************
Requesting subset of files:
['/clusterfs/minos/uploaded_data/bquiter/5d499b883d28ed775fe37cd5_Independent Review Slides/Independent Review Slides/Talk 2 - 190801 Schubert Review_Task 3 Overview FINAL.pptx', '/clusterfs/minos/uploaded_data/bquiter/5d499b883d28ed775fe37cd5_Independent Review Slides/Independent Review Slides/Talk 5 - Schubert Review Task 3.1.pptx']


We're now ready to instantiate a files query. For simplicity, we're simply asking for all the files in this case but in a real use-case you would select a subset of files from the returned file list in the previous step. We then do progress checks on this which will block until the query has completed. 

Following this we call `save_file` which will store the file to the user's JupyterHub home directory for later browsing. Note, for larger queries, this step might take some time.

In [12]:
file_query_id = api.start_files_query(files=file_list_for_desired_coll[0:2])
print ('Submitted files query request with query ID {0}'.format(file_query_id))
run_query_progress_checks(file_query_id)

print ('******************************')
print ('Moving files query results to home directory')
result = api.save_file(file_query_id)
print (result['message'])

Submitted files query request with query ID 5e4dc550f82105381e7800be
Checking query progress every 3 seconds for query 5e4dc550f82105381e7800be
Current query progress: 100%
******************************
Moving files query results to home directory
Successfully moved query results to home directory. Filename is: download_5e4dc550f82105381e7800be.zip


Alternatively, we can start a query but using a datacollection name instead of a list of files. This is demonstrated below. Note that datacollection queries could produce very large outputs (possibly hundreds of gigabytes of data). This might take 1-2 hours to complete depending on how much data is requested.

In [13]:
print ('******************************')
print ('Requesting datacollection:')
print (desired_coll)
coll_query_id = api.start_datacollection_query(datacollection=desired_coll)
print ('Submitted datacollection query request with query ID {0}'.format(coll_query_id))    
run_query_progress_checks(coll_query_id)

print ('******************************')
print ('Moving datacollection query results to home directory')
result = api.save_file(coll_query_id)
print (result['message'])   

******************************
Requesting datacollection:
NA-22 Independent Review_2019-08-06T08:00:00-08:00-to-2019-08-06T15:30:00-08:00
Submitted datacollection query request with query ID 5e4dc560f82105375549dcbe
Checking query progress every 3 seconds for query 5e4dc560f82105375549dcbe
Current query progress: 100%
******************************
Moving datacollection query results to home directory
Successfully moved query results to home directory. Filename is: download_5e4dc560f82105375549dcbe.zip


If the Python API is running locally, we can download results directly to disk. By default, if the `jupyterhub` flag is not provided, the API attempts to save files locally. In this case, a `local_path` flag must be provided. If `jupyterhub` is `True`, `local_path` is ignored.

In [ ]:
print ('******************************')
print ('Downloading datacollection query results to local disk')
result = api.save_file(coll_query_id, jupyterhub=False, local_path='/tmp')
print (result['message'])